In [2]:
from influxdb import InfluxDBClient
import time;
import random

json_body = []

for j in range(1, 20):
    for x in range(1, 100):
        time.sleep(0.1)
        ticks = round(time.time()*1000) # time in ms
        print(ticks)
        json_body += [
            {
                "measurement": "kek",
                "time": ticks*1000*1000,
                "fields": {
                    "value": random.random(),
                    "value2": random.random()
                }
            }
        ]
    
    client = InfluxDBClient('localhost', 8086, 'root', 'root', 'statsdemo')
    client.write_points(json_body)

1579259547908
1579259548008
1579259548110
1579259548210
1579259548311
1579259548411
1579259548512
1579259548613
1579259548713
1579259548814
1579259548914
1579259549015
1579259549116
1579259549217
1579259549317
1579259549418
1579259549518
1579259549618
1579259549719
1579259549820
1579259549920
1579259550021
1579259550121
1579259550221
1579259550321
1579259550422
1579259550522
1579259550622
1579259550723
1579259550824
1579259550924
1579259551025
1579259551125
1579259551226
1579259551327
1579259551427
1579259551528
1579259551629
1579259551729
1579259551830
1579259551931
1579259552032
1579259552132
1579259552233
1579259552333
1579259552434
1579259552535
1579259552635
1579259552737
1579259552837
1579259552938
1579259553039
1579259553139
1579259553240
1579259553340
1579259553440
1579259553541
1579259553641
1579259553741
1579259553842
1579259553943
1579259554043
1579259554144
1579259554244
1579259554346
1579259554446
1579259554547
1579259554647
1579259554748
1579259554848
1579259554949
157925

ConnectionError: HTTPConnectionPool(host='localhost', port=8086): Max retries exceeded with url: /write?db=statsdemo (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000014F38FE56A0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it',))

In [ ]:
import json
import sys
import matplotlib.pyplot as plt
%matplotlib inline  
file = open("log18.txt")

def flatten(data, prefix=""):
    flat = dict()
    
    if isinstance(data,dict):
        if prefix:
            prefix += "."
        for k,v in data.items():
            flat.update(flatten(v, prefix + str(k)))
        return flat;
    
    if isinstance(data,list):
        if prefix:
            prefix += "."
        i = 0;
        for v in data:
            flat.update(flatten(v, prefix + str(i)))
            i += 1
        return flat
    
    return {prefix:data}



json_body = []
start_time = round(time.time()*1000) # time in ms

id = []
stack = []
bat = {"cellA":[],"cellB":[]}
for line in file:
    try:
        data = json.loads(line)
        if data["id"] == "sensor":
            try:
                
                flat = flatten(data)
                flat["id"] = 0.0 #Need to handle enumerated states!
                flt = dict()
                
                for key in flat:
                    flt[key] = float(flat[key])
                    #print(flat)
                
                json_body += [
                    {
                        "measurement": "flight1",
                        "time": (start_time + data[u"tick"])*1000*1000,
                        "fields": flt
                    }
                ]
                
                
                
            except KeyError:
                print('keyError')
                print(line)
        if data[u"id"] == u"status":
            tick_stat.append(data[u"tick"])
            #bat["cell"].append(data[u'bat'][u"cell"])
    except ValueError:
        print('line not JSON')
file.close()

client = InfluxDBClient('localhost', 8086, 'root', 'root', 'statsdemo')
client.write_points(json_body)
